In [ ]:
import pandas as pd

# Load manager daily returns (decimal, e.g. 0.001 = 0.10%)
df = pd.read_csv("../data/manager_returns.csv", parse_dates=["date"]).sort_values("date").set_index("date")
returns_raw = df["return"].astype(float).dropna()
returns_raw.head()


In [ ]:
# Load benchmark prices and convert to daily returns
bench = pd.read_csv("../data/benchmark_prices.csv", parse_dates=["date"]).sort_values("date").set_index("date")
benchmark_raw = bench["price"].astype(float).pct_change().dropna()

# Load daily risk-free returns (decimal daily return)
rf = pd.read_csv("../data/risk_free_daily.csv", parse_dates=["date"]).sort_values("date").set_index("date")["rf"].astype(float).dropna()

# Align on common dates
common_idx = returns_raw.index.intersection(benchmark_raw.index).intersection(rf.index)
returns_raw = returns_raw.reindex(common_idx)
benchmark_raw = benchmark_raw.reindex(common_idx)
rf = rf.reindex(common_idx)

# Compute excess returns for risk-adjusted metrics
returns = returns_raw - rf
benchmark_rets = benchmark_raw - rf

returns.tail(), benchmark_rets.tail()


In [ ]:
import pyfolio as pf

# Tear sheet uses *excess returns* when you want a non-zero risk-free rate
pf.create_full_tear_sheet(
    returns,
    benchmark_rets=benchmark_rets
)


In [ ]:
# Key performance statistics (computed on excess returns)
stats = pf.timeseries.perf_stats(returns, factor_returns=benchmark_rets)
stats.to_frame("value")


In [ ]:
import numpy as np

APPROX_TRADING_DAYS = 252

rolling_vol = returns.rolling(63, min_periods=63).std() * np.sqrt(APPROX_TRADING_DAYS)
rolling_sharpe = (returns.rolling(252, min_periods=252).mean() / returns.rolling(252, min_periods=252).std()) * np.sqrt(APPROX_TRADING_DAYS)

rolling_vol.dropna().tail(), rolling_sharpe.dropna().tail()


In [ ]:
# Monitoring snippets (raw returns are usually used for "what happened", while excess returns for Sharpe/alpha)
worst_days = returns_raw.nsmallest(10)
worst_months = returns_raw.resample("ME").apply(lambda x: (1+x).prod()-1).nsmallest(10)

worst_days, worst_months
